In [ ]:
#Installing Androguard.
#Installed and commented
#!pip install -U androguard

     |████████████████████████████████| 931kB 4.6MB/s 
     |████████████████████████████████| 112kB 17.1MB/s 
  Found existing installation: pydot 1.3.0
    Uninstalling pydot-1.3.0:
      Successfully uninstalled pydot-1.3.0


In [ ]:
#Import libraries
from androguard.misc import AnalyzeAPK
import matplotlib.pyplot as plt
import numpy as np
from random import randrange
from IPython.display import clear_output
import json
import glob
import time
import sys
import cv2
import os

In [ ]:
#!pip install -U androguard
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#Resizing all channels to a fixed shape by using Nearest neighbour interpolation
def OneD2TwoD_resize(image_pixels):

    h = int(len(image_pixels)**0.5)
    w = int(len(image_pixels)/h)
    
    unused_valus = len(image_pixels) - h*w
    inc = 0
    if(unused_valus != 0):
        unused_valus = len(image_pixels) - h*w
        empty_spots = w-unused_valus
        i = unused_valus
        j = unused_valus+empty_spots
        image_pixels += image_pixels[-j:-i]
        inc+=1
    
    image_pixels = np.array(image_pixels)
    twoD_image = image_pixels.reshape(h+inc,w)
    
    twoD_image = twoD_image.astype('float32')

    if(twoD_image.shape[0]>64):
        image_resized = cv2.resize(twoD_image, (64,64), interpolation = cv2.INTER_NEAREST)# INTER_LINEAR
    else:
        image_resized = cv2.resize(twoD_image, (64,64), interpolation = cv2.INTER_NEAREST)# INTER_LINEAR
    return image_resized

In [ ]:
#Convert properties from classes.dex file to integers (pixel values) in range [0,255]
def dex2Image(dx,d):
    #Convert APIs to pixel values
    blue_channel_stuff, API_calls_pixels = API_calls2pixels(dx)
    
    #Convert Opcodes to pixel values
    opcodes_pixls = list(opcodes2pixels(dx))
    
    red_cahnnel = OneD2TwoD_resize(API_calls_pixels + opcodes_pixls)  #[API_calls_pixels + opcodes_pixls][0]

    #Convert Opcodes to pixel values
    opcodes_pixls = list(opcodes2pixels(dx))
    #blue_channel_stuff += opcodes_pixls
    
    
    #Extract malicious strings
    blue_channel_stuff += strings2pixels(d)
    
    return red_cahnnel, blue_channel_stuff

#Convert API calls to pixel values
def API_calls2pixels(dx):
    imprtnt_API_calls = []
    nrml_API_calls = []
    for api in dx.get_external_classes():
        api_tmp = api.get_vm_class()
        for i in api_tmp.get_methods():
            api_call = str(i).split('(')[0] # ignoring parameters and return type.
            if(api_call in API_calls_dict):
                imprtnt_API_calls.append(API_calls_dict[api_call])
            else:
                nrml_API_calls.append(sum(api_call.encode())%256)
    return imprtnt_API_calls, nrml_API_calls

#Convert opcodes to pixel values
def opcodes2pixels(dx):
    opcodes = []
    for method in dx.get_methods():
        if method.is_external():
            continue
        m = method.get_method()
        for ins in m.get_instructions():
          #if(ins.get_op_value not in opcodes):
          opcodes.append(ins.get_op_value())
    opcodes = list(set(opcodes))
    return opcodes

#Convert protected strings to pixel values
def strings2pixels(d):
    string_pixels = []
    all_strings = d[0].get_strings()
    
    for strng in all_strings:
        if(strng in API_calls_dict):
            string_pixels.append(API_calls_dict[strng])
        elif(strng in malicious_permissions):
            string_pixels.append(malicious_permissions[strng])
        elif(strng in malicous_activities):
            string_pixels.append(malicous_activities[strng])
        elif(strng in malicious_services):
            string_pixels.append(malicious_services[strng])
        elif(strng in malicious_recivers):
            string_pixels.append(malicious_recivers[strng])
        elif(strng in malicious_intents):
            string_pixels.append(malicious_intents[strng])
        else:
            pass
    #[string_pixels.append((sum(strng.encode()))%256) for strng in all_strings]
    return string_pixels

In [ ]:
#Convert properties from AndroidMAnifest.xml file to pixel values
def manifest2Image(a):
    #Permissions
    permissions = a.get_permissions() + a.get_declared_permissions()
    #print(permissions)
    imprtnt_permissions = []
    nrml_permissions = []
    for permission in permissions:
        perm = permission.split('.')[-1]
        # main key word (permission) is oftenly(always) comes at last.
        if(perm in malicious_permissions):
            imprtnt_permissions.append(malicious_permissions[perm])
        elif(perm in decleared_permissions):
            imprtnt_permissions.append(decleared_permissions[perm])
        else:
            nrml_permissions += list(perm.encode())
    
    #Activities
    activities = a.get_activities()
    imprtnt_activities = []
    nrml_activities = []
    for activity in activities:
        act = activity.split('.')[-1]
        if(act in malicous_activities):
            imprtnt_activities.append(malicous_activities[act])
        else:
            nrml_activities += list(act.encode())
            
    #Services
    services = a.get_services()
    imprtnt_services = []
    nrml_services = []
    for service in services:
        srvc = service.split('.')[-1]
        if(srvc in malicious_services):
            imprtnt_services.append(malicious_services[srvc])
        else:
            nrml_services += list(srvc.encode())
            
    #Recivers
    receivers = a.get_receivers()
    imprtnt_receivers = []
    nrml_receivers = []
    for receiver in receivers:
        recevr = receiver.split('.')[-1]
        if(recevr in malicious_recivers):
            imprtnt_receivers.append(recevr)
        else:
            nrml_receivers += list(recevr.encode())
            
    #Providers
    providers = a.get_providers()
    nrml_providers = []
    for provider in providers:
        nrml_providers += list(provider.encode())
        
    #Intents
    imprtnt_intents = []
    nrml_intents = []
    manifest_list = {'permissions':permissions,'activity' : activities, 'service': services, 'receiver':receivers, 'provider':providers}
    intents_itemtype = {'activity' : activities, 'service': services, 'receiver':receivers, 'provider':providers}
    for itemtype, listt in intents_itemtype.items():
        for item in listt:
            try:
                for intnts in a.get_intent_filters(itemtype, item).values():
                    for intnt in intnts:
                        if(intnt in malicious_intents):
                            imprtnt_intents.append(malicious_intents[intnt])
                        else:
                            nrml_intents += list(intnt.encode())
            except:
                pass
    blue_channel_stuff = imprtnt_permissions + imprtnt_activities + imprtnt_services + imprtnt_intents + imprtnt_receivers
    green_channel = nrml_permissions + nrml_activities + nrml_services + nrml_receivers + nrml_providers + nrml_intents
    green_channel = OneD2TwoD_resize(green_channel)
    return green_channel, blue_channel_stuff

In [ ]:
#Collecting properties (pixel values) from files, place on channel, resize the channel and merge to make an image
def apk2image(a, d, dx):
    green_channel, blue_channel_stuff1 = manifest2Image(a)
    
    red_channel, blue_channel_stuff2 = dex2Image(dx, d)
    
    blue_channel = blue_channel_stuff1 + blue_channel_stuff2
    blue_channel = OneD2TwoD_resize(blue_channel)

    image = cv2.merge((red_channel, green_channel, blue_channel))
    image = image.astype(dtype='uint8')
    return image

## Single Processing

In [ ]:
shas_list = []
for line in open('/Users/asim/Desktop/Asim_Research/Adversarial_ML/AndroZoo/APK_Files/Malware/top_5_families_APKs', 'r'):
    shas_list.append(line.rstrip())
    
shas_list = [os.path.join(sha + '.apk') for sha in shas_list]

This cell is added.

In [ ]:
#Reading APK's SHA256 from downloaded APKs
malware_imges_path = '/Users/asim/Desktop/Asim_Research/Adversarial_ML/AndroZoo/APK_Files/Malware/Top_5_Families_New_Approach/'
converted_imges = os.listdir(malware_imges_path)
converted_shas = [sha[:-4] for sha in converted_imges]
print(len(converted_imges), 'are converted.')

shas_list = list(set(shas_list) - set(converted_shas))
print(len(shas_list), 'APKs are remaining to be converted.')

In [ ]:
#main loop for converting an APK to image
malware_imges_path = '/Users/asim/Desktop/Asim_Research/Adversarial_ML/AndroZoo/APK_Files/Malware/Top_5_Families_New_Approach/'
malware_APKs_path = '/Volumes/EXTERNAL_AS/Malware_APK_V1/'

for sha in shas_list:
    print('Analysing :', sha)
    
    try:
        #Androguard
        a,d,dx = AnalyzeAPK(os.path.join(malware_APKs_path + sha))

        img = apk2image(a,d,dx)

        #Saving to local directory
        cv2.imwrite(os.path.join(malware_imges_path + sha[:-4] + '.jpg'), img)
    except:
        pass
    

**Multi Processing via threads for same APK to image loop)**

In [ ]:
imges_path = '/content/gdrive/My Drive/Adverserial_ML/AndroZoo/APKs_and_Images/Benign_Images/Androguard_and_PreDefined_Values/Inter_Linear/Ben_imgs_16K_onwards_interLinear/'
APKs_path = '/content/gdrive/My Drive/Adverserial_ML/AndroZoo/APKs_and_Images/Benign_APKs_16K_Onwards/'

def apk2image_multi(sha):
    apk = os.path.join(APKs_path + sha + '.apk')
    print('Analysing :', apk[-68:])
    
    try:
        #Androguard
        a,d,dx = AnalyzeAPK(apk)

        img = apk2image(a,d,dx)

        #Saving to local directory
        cv2.imwrite(os.path.join(imges_path + sha + '.jpg'), img)
    except:
        pass
    

In [ ]:
shas_path = '/content/gdrive/My Drive/Adverserial_ML/AndroZoo/APKs_and_Images/Benign_APKs_16K_Onwards/'
shas_list = os.listdir(shas_path)
shas_list = [sha[:-4] for sha in shas_list]
print(len(shas_list))
print(shas_list[0])


converted_imgs_path = '/content/gdrive/My Drive/Adverserial_ML/AndroZoo/APKs_and_Images/Benign_Images/Androguard_and_PreDefined_Values/Inter_Linear/Ben_imgs_16K_onwards_interLinear/'
converted_imgs = os.listdir(converted_imgs_path)
converted_imgs_list = [sha[:-4] for sha in converted_imgs]
print(len(converted_imgs_list))
print(converted_imgs_list[0])
apks_list = list(set(shas_list) - set(converted_imgs_list))

print(len(apks_list))

13898
00F39952259EBEEE789C1EC73C21B32720B4E0B7B37B8E45D45D5B0FEA9A89C3
13732
00B6686710971A60339541F57DA985C8C5CD7D0ACE38C7F705F79DBBFF8D333D
166


In [ ]:
apks_list = []
for sha in shas_list:
  if(sha not in converted_imgs_list):
    apks_list.append(sha)
print(len(apks_list))

166


In [ ]:
print(apks_list[:5])
print(apks_list[2000:2005])
print(apks_list[4000:4005])

['00F1DAA527AEBB4F1DDEB6204642E08D49769DDC29238BD6BA73EF9D2D0E1862', '00B2288DBEDCE4D0756AD6766B93F6CD4A4F67519587D69DF4B769C27EB7E76F', '00B66CE4C010A81D47C4C5A855772C8E29B6687E897353E554235504B5D624C2', '00C9120CDD837820C522A6413F776CB3B2538422D5CF1888BA1E7B22D997B5F8', '00DD62D4F700CF4CEFD7330737AD554824474841D04A71D1F1D0F86B96244EB9']
['00E1FA91B134D8BDD5F16652F7CA6BA35CBD0D3B68C2F42833512CD7152A43A6', '00DDD0301F2ED1118588920D3802C49B7C14275AF9FD9F774F566691D321B2C5', '009BBC923A30DEF661A7C46282EA6B1FF83FE0452AD44DA11E14FD4B01CB8812', '00BF7F976083E7B68C9D2F5AAF425A717C1E602C65B61744E47389EA3528022D', '00E3AD725A8F7050553DBCC8C848B8434F02CEE9F340983AF3AFD5CD650F918D']
[]


In [ ]:
import concurrent.futures
start_time = time.time()

with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(apk2image_multi, apks_list[:100])
    
print(time.time()-start_time)

Analysing : 00F1DAA527AEBB4F1DDEB6204642E08D49769DDC29238BD6BA73EF9D2D0E1862.apk
Analysing : 00B2288DBEDCE4D0756AD6766B93F6CD4A4F67519587D69DF4B769C27EB7E76F.apk
Analysing : 00B66CE4C010A81D47C4C5A855772C8E29B6687E897353E554235504B5D624C2.apk
Analysing : 00C9120CDD837820C522A6413F776CB3B2538422D5CF1888BA1E7B22D997B5F8.apk
Analysing : 00DD62D4F700CF4CEFD7330737AD554824474841D04A71D1F1D0F86B96244EB9.apk
Analysing : 009DEEF065F3F622C9BBC70B989D5F8B34BA5AF985A87C7989EDED2AAFF4490D.apk
Analysing : 01075F1E5C56C91E7532FDEFD3DD27EEA2DF76DF9069A218E77F41AD4041375F.apk
Analysing : 009BBD4BE2AAEF0CB8AAA0C5849732A80FE07453BB2F15EFC0A1005B4F5E1552.apk
Analysing : 00F76CAA3F522F6447D50E3AC07C424F35C774867952164A7CF1659246336FDB.apk
Analysing : 0094ABDD1DD80E27F5CFFEB8312B98882D49FCB2F7C3D1CBA0C6022C7F3DB5BA.apk
Analysing : 00B10B915A15074B2A9834144165D9C472B7D0DCA74F7A0A98E00E2D217C2967.apk
Analysing : 00BF8634903AA6006AD61617237C89513E5C6250A2506E55D4C6C738E55C3BC7.apk
Analysing : 00B3332E3D51BDD0

Requested API Level could not be found, using 10 instead


Analysing : 00FA4F41C2409DE6CFACA511506E6C1081857976E29FE376B1E0B153268645EB.apk
Analysing : 008E414475237473A268A6D73BDD46D9F377ECCBC69D41A8BD6E9F18C3DAB73B.apk
Analysing : 00A76EF5CFF2587A7E5FC30E124C9F94F026FA85410B117652372835CF6056F3.apk
Analysing : 00B981B97CD23FD3FDB5276234ACA4C24B52731C07E512EFE8F27575A17F7D0A.apk
Analysing : 00EAF9F8E0320E53499AA4C2A898DF29FBAC51C9138EB3A2DD082B3ED5AA2C1B.apk
Analysing : 00F83D0C94C5F173BAEE2C0860FA042D07E406562B350A997161FD3944CF2744.apk
Analysing : 00E2471AAC9E0A51267727DB41A15CBA4A61620C0846B03122AA03C8124DB814.apk
Analysing : 0104A9CD91C8CA399B1728AF04C3D9A5F543017028CE12B2C935AF89BDA8BEDB.apk
Analysing : 00C788905A87573CDBD087DC5D5A6A11C9A9B71B4E399872FC738B63E425D0FD.apk
Analysing : 00ABA848BC1811FE9F73DE6EA4954FFFA648F40BA272C2617000198902B643FF.apk
Analysing : 00AB7D45D6A1C5F8C74D213E517678EA62F3C60A7795A8D16753679B0C3CD7B3.apk
Analysing : 00E45A8062D2634360E7FA2D0BA243EDCFD79A2556742E306275646B4AC30281.apk
Analysing : 00AD58AE0F0C487A

Requested API Level could not be found, using 10 instead


Analysing : 00D574DE8A4180B0E9A27ABC255F51D52E2CD0C80320819CF5AFF2D00848A194.apk
Analysing : 00BF7F976083E7B68C9D2F5AAF425A717C1E602C65B61744E47389EA3528022D.apk
Analysing : 00FCCAC4957369CD9E44B64BAC50399043AD9D967E52B8D862AFB4C2445AF700.apk
Analysing : 00C7A2F51187099A39D1EB475F5A7100D0798CE7CD027ED1D1B532FB5D77CE25.apk
Analysing : 00DA20D2961B2D8AB9BD224ABE0EFCC0418692E73335EC39C084BC41ABBB13BE.apk
Analysing : 00DABC99417AE2A4C120D143772C60EA034EE2A196C80C2ECDBFAD4B0B9000A9.apk
89.42275619506836


### **Making .npy file**
converting images to numpy array (.npy file) for training deep learning models


In [ ]:
#converting images to numpy array (.npy file) for training deep learning models
families_path = '/home/Asim/Adverserial_ML/AndroZoo/Malware/Images_by_HIM/Families_Images_correct_namings/Families_Images_geq15/*'

families_npy_path = '/home/Asim/Adverserial_ML/AndroZoo/Malware/Images_by_HIM/Families_Images_correct_namings/Families_npy_geq15/'
from numpy import load
count = 0
for family in glob.glob(families_path):
    family_name = family.split('/')[-1]
    if(len(os.listdir(family)) >= 2400):
        array = []
        print(family_name, len(os.listdir(family)))
        for img in glob.glob(os.path.join(families_path[:-1] + family_name + '/*')):
            array.append(plt.imread(img))

        np.save(os.path.join(families_npy_path + family_name + '.npy'), array[:2400])

**Predefined Properties**

# Important API Calls

In [ ]:
# Risky API_calls and methods
API_calls_dict = {
    'Ljavax/sql/ConnectionEvent;-><init>':250,
    'Ljava/nio/channels/WritableByteChannel;->close':240,
    'Landroid/service/carrier/CarrierService;->stopSelf':230,
    'Landroid/opengl/Matrix;->getClass':220,
    'Landroid/view/ViewStructure;->setCheckable':210,
    'Landroid/text/method/BaseKeyListener;->getInputType':200,
    'Landroid/provider/MediaStore$Images$Media;->wait':190,
    'Ljava/lang/Runtime;->exec':180,
    'Ljava/lang/System;->loadLibrary':180,
    'Landroid/widget/AdapterView;->refreshDrawableState':170,
    'Landroid/widget/MultiAutoCompleteTextView;->saveHierarchyState':170,

    'Ljava/io/BufferedOutputStream;-><init>':160,
    'Ljava/io/FileOutputStream;-><init>':160,

    'Landroid/app/PendingIntent;->send':150,
    'Landroid/app/AlarmManager;->Set':150,
    'Landroid/app/NativeActivity;->getVolumeControlStream':140,
    'Landroid/app/AcitivityManager;->killBackgroudProcess':140,
    
    'Landroid/content/pm/PacakageManager;->removePackageFromPrefe':130,
    'Landroid/content/pm/PacakageManager;->getInastallerPackageName':130,
    'Landroid/content/pm/PacakageManager;->getInstalledPackages':130,
    'Landroid/content/pm/PacakageManager;->getInstalledApplications':130,
    'Landroid/content/Intent;->startActivity':120,
    'Landroid/content/Intent;->getAction':120,
    'Landroid/content/Intent;->setDataAndType':120,
    'Landroid/content/ContentResolver;->delete':115,
    'Landroid/content/ContentResolver;->update':115,
    'Landroid/content/ContentResolver;->insert':115,
    'Landroid/content/Context;->getPackageManager':115,
    'Landroid/content/BroadcastReceiver;->abortBroadcast':115,

    'Landroid/database/sqlote/SQLiteDatabase;->execSQL':110,

    'Landroid/media/MediaRecorder;->MediaRecorder':100,
    'Ljava/net/HttpURLConnection;->connect':95,
    'Ljava/net/URLConnection-;>connect':95,
    'Lorg/apache/http/impl/client;->DefaultHttpClient':90,

    'Ljavax/crypto/Cipher;->getInstance':85,
    'Ljavax/crypto/Cipher;->Init':85,
    'Ljavax/crypto/Cipher;->doFinal':85,

    'Landroid/telephony/TelephonyManager;->getDataActivity':80,
    'Landroid/telephony/TelephonyManager;->getCallState':80,
    'Landroid/telephony/TelephonyManager;->getLine1Number':80,
    'Landroid/telephony/TelephonyManager;->getCellLocation':80,
    'Landroid/telephony/TelephonyManager;->getSubscriberId':80,
    'Landroid/telephony/TelephonyManager;->getDeviceId':80,
    'Landroid/telephony/TelephonyManager;->getNetworkType':80,
    'Landroid/telephony/TelephonyManager;->getSimOperator':80,
    'Landroid/telephony/TelephonyManager;->getSimSerialNumber':80,
    'Landroid/telephony/TelephonyManager;->getSimState':80,
    'Landroid/telephony/TelephonyManager;->getSubscriberId':80,
    'Landroid/telephony/SmsManager;->sendTextMessage':75,
    'Landroid/telephony/SmsManager;->sendMultipartTextMessage':75,
    'Landroid/telephony/SmsManager;->sendDataMessage':75,
    'Landroid/telephony/SmsMessage;->getServiceCenterAddress':75,
    'Landroid/telephony/gsm/SmsManager;->sendMultipartTextMessage':70,
    'Landroid/telephony/gsm/SmsManager;->sendDataMessage':70,
    'Landroid/telephony/gsm/SmsManager;->sendTextMessage':70,
    'Landroid/telephony/gsm/SmsManager;->getDisplayOriginatingAddress':70,
    'Landroid/telephony/gsm/SmsManager;->getDisplayMessageBody':70,
    'Landroid/telephony/PhoneStateListener;->onCallStateChanged':65,

    'Ldalvik/system/DexClassLoader;->loadClass':60,
    'Ldalvik/system/PathClassLoader;->loadClass':60,
    'Landroid/net/wifi/WifiInfo;->getSupplicantState':50,
    'Landroid/net/wifi/WifiManager;->getConnectionInfo':40,
    'Landroid/ContentResolver;->query':30, 
    'Landroid/location/LocationManager;->getLastKnownLocation':20,
}

## Collected from following Refrences

Android Malware Identification Through Visual Exploration of Disassembly Files. 2017
International Journal of Network Security, Vol.20, No.6, PP.1061-1073, Nov. 2018 (DOI: 10.6633/IJNS.201811 20(6).07) 1061

In [ ]:
# Risky API_calls and methods
malicious_apis_1 = {'Ljava/lang/Runtime->exec': 220, 'Landroid/content/Intent->startActivity': 210,
                 'Landroid/app/PendingIntent->send': 200, 'Landroid/app/AlarmManager->Set': 200,
                 'Landroid/content/pm/PacakageManager->removePackageFromPrefe': 190,
                 'Landroid/database/sqlote/SQLiteDatabase->execSQL': 180,
                 'Landroid/content/ContentResolver->delete': 170,
                 'Landroid/app/AcitivityManager->killBackgroudProcess': 160,
                 'Landroid/media/MediaRecorder->MediaRecorder': 150, 'Ljava/net/HttpURLConnection->connect': 140,
                 'Ljava/net/URLConnection->connect': 140, 'Lorg/apache/http/impl/client->DefaultHttpClient': 140,
                 'Landroid/content/BroadcastReceiver->abortBroadcast': 63,
                 'Landroid/telephony/PhoneStateListener->onCallStateChanged': 130,
                 'Landroid/content/Intent->getAction': 120, 'Ljavax/crypto/Cipher->getInstance': 110,
                 'Ljavax/crypto/Cipher->Init': 110, 'Ljavax/crypto/Cipher->doFinal': 110,
                 'Landroid/telephony/TelephonyManager->getLine1Number': 100,
                 'Landroid/content/pm/PacakageManager->getInastallerPackageName': 90,
                 'Landroid/content/pm/PacakageManager->getInstalledPackages': 90,
                 'Landroid/content/pm/PacakageManager->getInstalledApplications': 90,
                 'Landroid/location/LocationManager->getLastKnownLocation': 80,
                 'Landroid/telephony/TelephonyManager->getCellLocation': 80,
                 'Landroid/telephony/TelephonyManager->getSubscriberId': 71,
                 'Landroid/telephony/TelephonyManager->getDeviceId': 70,
                 'Landroid/telephony/SmsManager->sendTextMessage': 64,
                 'Landroid/telephony/gsm/SmsManager->sendMultipartTextMessage': 62,
                 'Landroid/telephony/SmsManager->sendMultipartTextMessage': 62,
                 'Landroid/telephony/gsm/SmsManager->sendDataMessage': 61,
                 'Landroid/telephony/gsm/SmsManager->sendTextMessage': 61,
                 'Landroid/telephony/SmsManager->sendDataMessage': 61,
                 'Landroid/telephony/gsm/SmsManager->getDisplayOriginatingAddress': 60,
                 'Landroid/telephony/gsm/SmsManager->getDisplayMessageBody': 60,
                 'Ldalvik/system/DexClassLoader->loadClass': 50, 'Ldalvik/system/PathClassLoader->loadClass': 50,
                 'Landroid/content/ContentResolver->update': 40, 'Landroid/content/ContentResolver->insert': 30,
                 'Landroid/ContentResolver->query': 20, 'Landroid/content/Intent->setDataAndType': 10
                }

Android Malware Detection based on Useful API Calls and Machine Learning 2018 IEEE First International Conference on Artificial Intelligence and Knowledge Engineering (AIKE)

In [ ]:
# you have to cehck whether Aandrogurad call method via '.' or '->'
malicious_apis_2 = {'Ljavax/sql/ConnectionEvent;-><init>':255, 'Ljava/nio/channels/WritableByteChannel;->close':250,
 'Landroid/service/carrier/CarrierService;->stopSelf':245 ,
 'Landroid/app/NativeActivity;->getVolumeControlStream':240, 
 'Landroid/widget/AdapterView;->refreshDrawableState':235, 'Landroid/opengl/Matrix;->getClass':230, 
 'Landroid/widget/MultiAutoCompleteTextView;->saveHierarchyState':225, 
 'Landroid/view/ViewStructure;->setCheckable':220, 'Landroid/text/method/BaseKeyListener;->getInputType':215, 
 'Landroid/provider/MediaStore$Images$Media;->wait':210
}

benign_api_1 = {'Ljava/lang/Object;.<init>':0, 'Ljava/lang/StringBuilder;.toString':5, 'Ljava/lang/StringBuilder;.append':10,
 'Landroid/app/Activity;.<init>':15, 'Ljava/lang/StringBuilder;.<init>':20, 'Ljava/lang/String;.equals':25,
 'Ljava/util/ArrayList;.<init>':30, 'Ljava/util/Iterator;.next':35,
 'Ljava/util/Iterator;.hasNext':40
}

Feature extraction from Android application packages and its usage in machine learning for malware classification. Computer Science Institute of Charles University 2017

In [ ]:
malicious_apis_3 = {'Ljava.io.BufferedOutputStream;-><init>':205, 'Ljava.io.FileOutputStream;-><init>':200,
 'Ljava.lang.System;->loadLibrary':195, 
 'Landroid.content.Context;->getPackageManager':190
}

Mining API Calls and Permissions for Android Malware Detection (2014)
The LNM Institute of Information Technology, Jaipur, India 

In [ ]:
malicious_apis_4 = {'Landroid/telephony/TelephonyManager;->getCallState':185,
 'Landroid/telephony/TelephonyManager;->getDataActivity':180,
 'Landroid/telephony/TelephonyManager;->getDeviceId':175,
 'Landroid/telephony/TelephonyManager;->getLine1number':170,
 'Landroid/telephony/TelephonyManager;->getNetworkType':165,
 'Landroid/telephony/TelephonyManager;->getSimOperator':160,
 'Landroid/telephony/TelephonyManager;->getSimSerialNumber':155,
 'Landroid/telephony/TelephonyManager;->getSimState':150,
 'Landroid/telephony/TelephonyManager;->getSubscriberId':145
}

## Important Permissions

Android Malware Detection Using Fine-Grained Features 
Hindawi Scientific Programming Volume 2020

In [ ]:
#Activity
malicous_activities = {
    'ACCESS_COARSE_LOCATION':250,
    'ACCESS_FINE_LOCATION':240,
    'READ_SMS':230,
    'SEND_SMS':220,
    'READ_PHONE_STATE':210,
    'READ_CONTACTS':200,
    'READ_HISTORY_BOOKMARKS':190,
    'MOUNT_UNMOUNT_FILESYSTEMS':180,
    'RECEIVE_SMS':170
}



In [ ]:
#Services
malicious_services = {
    'SEND_SMS':160,
    'READ_SMS':150,
    'ACCESS_COARSE_LOCATION':140,
    'ACCESS_FINE_LOCATION':130,
    'READ_PHONE_STATE':120
}


In [ ]:
#BroadCast Reecvier
malicious_recivers = {
    'SEND_SMS':110,
    'READ_SMS':100,
    'ACCESS_COARSE_LOCATION':90,
    'ACCESS_FINE_LOCATION':80,
    'READ_PHONE_STATE':70
}


In [ ]:
#Intents
malicious_intents = {
    'BOOT_COMPLETED':60,
    'SMS_RECEIVED':50,
    'BATTERY_CHANGE_ACTION':40,
    'MOUNT_UNMOUNT_FILESYSTEMS':30
}


In [ ]:
#Not used but decleared
decleared_permissions={
    'CAMERA':250,
    'RECORD_AUDIO':230,
    'READ_CONTACTS':210,
    'READ_CALL_LOG':200,
    'CALL_PHONE':190,
    'WRITE_CALL_LOG':180,
    'CHANGE_WIFI_STATE':170,
    'GET_ACCOUNTS':160,
    'READ_CALENDAR':140,
    'ACCESS_WIFI_STATE':130,
    'ACCESS_COARSE_LOCATION':120,
    'ACCESS_FINE_LOCATION':110,
    'READ_PHONE_STATE':90,
    'WRITE_EXTERNAL_STORAGE':80,
    'READ_EXTERNAL_STORAGE':70,
    'ACCESS_NETWORK_STATE':60,
    'WRITE_CALENDAR':50,
    'SEND_SMS':40,
    'READ_SMS':30,
    }

In [ ]:
#Permissions
malicious_permissions = {
    'CAMERA':250,
    'RECORD_AUDIO':240,
    'READ_CONTACTS':230,
    'READ_CALL_LOG':220,
    'CALL_PHONE':210,
    'WRITE_CALL_LOG':200,
    'CHANGE_WIFI_STATE':190,
    'READ_CALENDAR':180,
    'ACCESS_WIFI_STATE':170,
    'GET_ACCOUNTS':160,
    'WAKE_LOCK':150,
    'GET_TASKS':130,
    'VIBRATE':120,
    'PROCESS_OUTGOING_CALLS':110,
    'SYSTEM_ALERT_WINDOWS':100,
    'WRITE_SETTING':90,
    'WRITE_EXTERNAL_STORAGE':80,
    'READ_EXTERNAL_STORAGE':70,
    'WRITE_CALENDAR':60,
    'ACCESS_NETWORK_STATE':50,
    'READ_HISTORY_BOOKMARKS':40
}